In [1]:
import os
import shutil
import sys
import numpy as np
import scipy
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf
import cv2

S:\Softwares\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_path = os.getcwd() + "\\data\\fruits-360\\"

In [3]:
def read(data_path, folder_type='train'):
    if folder_type.lower() == 'train':
        path = data_path + "\\" + "Training"
    elif folder_type.lower() == 'test':
        path = data_path + "\\" + "Test"
    else:
        print("Wrong folder path")
        sys.exit(1)
    
    total_fruits_list  = os.listdir(path)
    total_fruits_count = len(total_fruits_list)
    
    
    
    all_images_count = 0
    
    total_fruits_list_temp = total_fruits_list[:5]
    total_fruits_count_temp = len(total_fruits_list_temp)
    
    for fruit in total_fruits_list_temp:        
        for img in os.listdir(path + "\\" + fruit):
            all_images_count += 1
            
    
    data = {}
    labels = {}
    data[folder_type] = np.zeros(shape=(all_images_count, 100, 100, 3), dtype=np.float64)
    labels[folder_type] = np.zeros(all_images_count)
    print(data[folder_type].shape)
    print(labels[folder_type].shape)
    
    data_counter = 0
    label_counter = 0
    
    label_to_idx_dict = {}
    idx_to_label_dict = {}
    
    for fruit in total_fruits_list_temp:        
        print(folder_type, " : ", fruit)  
        label_to_idx_dict[fruit] = label_counter
        idx_to_label_dict[label_counter] = fruit
        for img in os.listdir(path + "\\" + fruit):
            cur_img = cv2.imread(path + "\\" + fruit + "\\" + img)            
            data[folder_type][data_counter,:,:,:] = cur_img                        
            labels[folder_type][data_counter] = label_counter
            data_counter += 1
        label_counter += 1
            
    return data[folder_type], labels[folder_type], len(total_fruits_list_temp), label_to_idx_dict, idx_to_label_dict
    

In [4]:
training_data, training_labels, num_classes, \
    label_to_idx_dict_train,idx_to_label_dict_train = read(data_path, folder_type="train")

test_data, test_labels, num_classes, \
    label_to_idx_dict_test, idx_to_label_dict_test = read(data_path, folder_type="test")

(2449, 100, 100, 3)
(2449,)
train  :  Apple Braeburn
train  :  Apple Golden 1
train  :  Apple Golden 2
train  :  Apple Golden 3
train  :  Apple Granny Smith
(817, 100, 100, 3)
(817,)
test  :  Apple Braeburn
test  :  Apple Golden 1
test  :  Apple Golden 2
test  :  Apple Golden 3
test  :  Apple Granny Smith


In [ ]:
'''
hf_train_data = h5py.File('training_data.h5', 'w')
hf_train_labels = h5py.File('training_labels.h5', 'w')
hf_test_data = h5py.File('test_data.h5', 'w')
hf_test_labels = h5py.File('test_labels.h5', 'w')

hf_train_data.create_dataset('training_data', data=training_data)
asciiList = [n.encode("ascii", "ignore") for n in training_labels]
hf_train_labels.create_dataset('training_labels', (len(asciiList),1),'S25', asciiList)

hf_test_data.create_dataset('test_data', data=test_data)
asciiList_test = [n.encode("ascii", "ignore") for n in test_labels]
hf_test_labels.create_dataset('test_labels', (len(asciiList_test),1),'S25', asciiList_test)
'''

In [5]:
# Display the dimensions of the data
print("Shape of training data is {}".format(training_data.shape)) 
print("Shape of training labels is {}".format(training_labels.shape)) 

print("Shape of test data is {}".format(test_data.shape)) 
print("Shape of test labels is {}".format(test_labels.shape)) 

Shape of training data is (2449, 100, 100, 3)
Shape of training labels is (2449,)
Shape of test data is (817, 100, 100, 3)
Shape of test labels is (817,)


In [6]:
label_to_idx_dict_train

{'Apple Braeburn': 0,
 'Apple Golden 1': 1,
 'Apple Golden 2': 2,
 'Apple Golden 3': 3,
 'Apple Granny Smith': 4}

In [16]:
# Represent labels as one hot encoding
training_labels_one_hot = tf.one_hot(training_labels, num_classes)
test_labels_one_hot = tf.one_hot(test_labels, num_classes)

In [17]:
with tf.Session() as sess:
    print(sess.run(training_labels_one_hot))
    print(sess.run(test_labels_one_hot))

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [18]:
# Normalize the input
training_data_norm = training_data/255.0
test_data_norm = test_data/255.0

print("Shape of normalized training data is {}".format(training_data_norm.shape)) 
print("Shape of normalized test data is {}".format(test_data_norm.shape)) 


Shape of normalized training data is (2449, 100, 100, 3)
Shape of normalized test data is (817, 100, 100, 3)


### Define the architecture 

1. Conv2D - 16 5 x 5 x 3 filters
2. Batch Norm
3. Relu
4. Max Pool - 2x2 Stride 2

1. Conv2D - 32 5x5x16 filters
2. Batch Norm
3. Relu
4. Max Pool - 2x2 Stride 2

1. Conv2D - 64 5x5x32 filters
2. Batch Norm
3. Relu
4. Max Pool - 2x2 Stride 2

1. Conv2D - 128 5x5x64 filters
2. Batch Norm
3. Relu
4. Max Pool - 2x2 Stride 2

1. Fully Connected - 1024
2. Fully Connected - 256 

1. Softmax  - 81


In [25]:
def initialize_parameters():
    
    parameters = {}
    
    parameters["W1"] = tf.get_variable(shape=(5,5,3,16), initializer=tf.contrib.layers.xavier_initializer(seed=0))
    parameters["W2"] = tf.get_variable(shape=(5,5,16,32), initializer=tf.contrib.layers.xavier_initializer(seed=0))
    parameters["W3"] = tf.get_variable(shape=(5,5,32,64), initializer=tf.contrib.layers.xavier_initializer(seed=0))
    parameters["W4"] = tf.get_variable(shape=(5,5,64,128), initializer=tf.contrib.layers.xavier_initializer(seed=0))
    
    return parameters
   

    

In [26]:
# Define 1 convolution block 
def conv2d_Block(X, W, s, padding='SAME'):
    
    Z = tf.nn.conv2d(X, W, stride=[1,s,s,1], padding=padding)
    
    A = tf.nn.relu(conv)
    
    return A
   
    

In [27]:
# Define max pool block
def maxpool2d_Block(X, f, padding='SAME'):

    max_pool = tf.nn.max_pool(X, [1,f,f,1], stride=[1,f,f,1], padding=padding)
    
    return max_pool

In [28]:
# Forward activation

def forward_pass(X, parameters):
    
    
    # Perform series of convolution operation
    C1 = conv2d_Block(X, parameters["W1"], 1, "SAME")
    M1 = maxpool2d_Block(C1, 2, "SAME")
    
    C2 = conv2d_Block(M1,  parameters["W2"], 1, "SAME")
    M2 = maxpool2d_Block(C2, 2,  "SAME")
    
    C3 = conv2d_Block(M2,  parameters["W3"], 1, "SAME")
    M3 = maxpool2d_Block(C3, 2,  "SAME")
    
    C4 = conv2d_Block(M3, parameters[" W4"], 1, "SAME")
    M4 = maxpool2d_Block(C4, 2, "SAME")
    
    # Flatten 
    P4 =  tf.contrib.layers.flatten(M4)
    
    # Fully connected - 1
    F1 = tf.contrib.layers.fully_connected(P4, 1024)
    
    # Fully connected - 2
    F2 = tf.contrib.layers.fully_connected(F1, 256)
    
    # last layer
    F3 = tf.contrib.layers.fully_connected(F2, 81, activation_fn=None)
    
    return F3
    
    
    


In [32]:
# Compute the cost

def compute_cost(logits, labels):

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))

    return cost

In [33]:
# Optimizer setup
def optimizer(alpha, cost):
    
    train = tf.train.AdamOptimizer(alpha).minimize(cost)
    
    return train

In [ ]:
# Create the model

def model(X, y, num_iterations=, mini_batch_size=, learning_rate=, ):
    
    
    sess = tf.Session()
    
    Z = forward_pass(X, parameters)
    
    cost = compute_cost(Z, y)
    
    train = optimizer(learning_rate, cost)
    
    init = tf.global_variables_initializer()
    
    sess.run(init)
    
    
    
    
    